In [1]:
from langchain import hub
prompt = hub.pull("tyfann/llm4commit-fewshot")

In [2]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    # api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.cn/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [3]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0)
    # completion = client.chat.completions.create(model="gpt-3.5-turbo-ca", messages=messages, temperature=0.5)
    return completion.choices[0].message.content

In [4]:
import json
from tqdm import tqdm

# Python part

In [5]:
with open('../data/angular_filtered/subsets/test_dev.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

In [6]:
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    # merged_diff = '\n'.join(diff['diff'] for diff in data['diff'])
    messages = prompt.invoke(
        {"DIFF": data['diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        gpt_msg.append("")

Processing documents: 100%|██████████| 603/603 [07:41<00:00,  1.31it/s]


In [7]:
for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_fewshot'] = msg
import os
# output_file = '../data/chronicle/rag_baseline/zeroshot/rag_baseline_python_chatgpt.json'
output_file = '../data/angular_filtered/subsets/generation/dev_test_gpt35_fewshot.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [8]:
output_file = '../data/angular_filtered/subsets/generation/dev_test_gpt35_fewshot.txt'
with open(output_file, 'w', encoding='UTF-8') as f:
    for item in org_data:
        f.write(item['chatgpt_fewshot'].replace('\n', '\\n').replace('\r', '\\r') + '\n')

In [9]:
def chatgpt_gen(org_data):
    gpt_msg = []
    for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
        # merged_diff = '\n'.join(diff['diff'] for diff in data['diff'])
        messages = prompt.invoke(
            {"DIFF": data['diff']}
        ).to_messages()
        example_prompt = [{'role': 'user','content': messages[0].content},]
        try:
            gpt_msg.append(gpt_35_api(example_prompt))
        except:
            print(index)
            gpt_msg.append("")

In [ ]:
with open('../data/final_preprocessed_data/js_baseline_test_data.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

org_data = org_data[:1000]
chatgpt_gen(org_data)

In [11]:

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt'] = msg
import os
# output_file = '../data/chronicle/rag_baseline/zeroshot/rag_baseline_python_chatgpt.json'
output_file = '../data/final_preprocessed_data/discriminator/dis_eval_data_chatgpt_finetune.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

# JavaScript part

In [17]:
with open('../data/scenario/filtered_data/project_based_react_multi_diff_filtered.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

org_data = org_data[:1000]
gpt_msg = []
for data in tqdm(org_data, total=len(org_data), desc="Processing documents"):
    merged_diff = '\n'.join(diff['diff'] for diff in data['diff'])
    messages = prompt.invoke(
        {"DIFFS": merged_diff}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt'] = msg

In [18]:
import os
# output_file = '../data/chronicle/rag_baseline/zeroshot/rag_baseline_js_chatgpt.json'
output_file = '../data/scenario/zeroshot/scenario_react_multi_diff_chatgpt_zeroshot.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)

## COT

In [7]:
from langchain import hub

prompt = hub.pull("tyfann/llm4commit-cot:7f1b7f53")
gpt_cot_msg = []

example_q = """"diff --git a/resources/lib/kodi/library_utils.py b/resources/lib/kodi/library_utils.py @@ -25,7 +25,7 @@ from resources.lib.utils.logging import LOG, measure_exec_time_decorator\nLIBRARY_HOME = 'library'\nFOLDER_NAME_MOVIES = 'movies'\nFOLDER_NAME_SHOWS = 'shows'\n-ILLEGAL_CHARACTERS = '[<|>|\"|?|$|!|:|#|*]'\n+ILLEGAL_CHARACTERS = '[<|>|\"|?|$|!|:|#|*|/|\\\\\\\\]'\ndef get_library_path():\n"""
example_a = """First, the scope of code changes is to add extra information to an existing field, and second, according to the filename "library_utils.py", it is a utils class, so there is no strong relationship between this file and the entire project. And for the modification it mainly adds the missing slash and backslash to ILLEGAL_CHARACTERS. So the commit message for the code diff is "Add missing slash backslash to illegal chars" """
for data in tqdm(org_data, total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"Example_Question": example_q, "Example_Answer": example_a, "Question": data['diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_cot_msg.append(gpt_35_api(example_prompt))

Processing documents: 100%|██████████| 1000/1000 [12:07<00:00,  1.38it/s]


In [12]:
len(gpt_msg), len(gpt_cot_msg)

In [13]:
import re

def preprocess(text):
    if re.search(r'commit\s*message', text, re.IGNORECASE):
        pattern = r'\"([^"]+)'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return text
    else:
        return text

In [8]:
for item, cot_msg in zip(org_data, gpt_cot_msg):
    item['chatgpt_cot'] = cot_msg

output_file = '../data/final_preprocessed_data/discriminator/dis_eval_data_chatgpt_cot.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [9]:
for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt_v3'] = msg

output_file = '../../bleu/all_result_chatgpt.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)